### Load modules and specify file path

In [ ]:
# Import required modules
import pandas as pd
import glob2
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Specify paths to files
raw_data_path = r'C:\Users\MT1070\Desktop\Master Call Volume\raw-report-extracts' 

# Display names of files in master folder
raw_files = glob2.glob(raw_data_path + "/*.xlsx")

# Initialize an empty data frame to store data from all files
final_sheet = pd.DataFrame()

### Read Data Set and Configure Entry Date 

In [ ]:
# Iteratively read data extracts from master files and append them to the final DataFrame
data_frames = []  # Store DataFrames from master files
for file in raw_files:
    try:
        df = pd.read_excel(file, sheet_name=None, skipfooter=2, engine='openpyxl')
        df_concat = pd.concat(df.values(), ignore_index=True, sort=False)
        data_frames.append(df_concat)
    except Exception as e:
        print(f"Error occurred while processing file: {file}")
        print(str(e))

## Reset Index and Set Column Headers

In [30]:
if len(data_frames) > 0:
    final_sheet = pd.concat(data_frames, ignore_index=True)

final_sheet.reset_index(drop=True, inplace=True)

# Define headers
final_sheet.columns = final_sheet.iloc[0]
final_sheet = final_sheet[1:]
# final_sheet.columns


Index(['callid', 'dnis', 'ani', 'datetime', 'internal_or_external', 'language',
       'department_utternace_first_try_success',
       'department_utternace_max_nomatch',
       'department_utternace_second_try_success',
       'department_utternace_third_try_success',
       'department_utternace_first_nomatch',
       'department_utternace_second_nomatch',
       'department_utternace_third_nomatch',
       'department_utternace_first_noinput',
       'department_utternace_second_noinput',
       'department_utternace_third_noinput',
       'department_utternace_max_noinput', 'department_name',
       'department_phone', 'call_transfer_success',
       'element_name_call_exited_in_cvp', 'department_users_says_no_1',
       'department_users_says_no_max'],
      dtype='object', name=0)

In [31]:
# Print the values in the "datetime" column
column_name = 'datetime'
if column_name.strip() in final_sheet.columns:
    datetime_column = final_sheet[column_name.strip()]
    print(f"Values in the '{column_name}' column:")
    print(datetime_column)
else:
    print(f"Column '{column_name.strip()}' not found in the DataFrame.")


Values in the 'datetime' column:
1         5/31/23 2:30:06 PM
2         5/31/23 7:45:03 PM
3        5/31/23 12:35:34 PM
4        5/31/23 11:59:46 AM
5         5/31/23 3:34:31 PM
                ...         
16538      6/6/23 3:27:23 PM
16539      6/6/23 4:24:39 PM
16540      6/6/23 2:44:27 PM
16541      6/6/23 2:36:08 PM
16542      6/6/23 9:29:29 AM
Name: datetime, Length: 16542, dtype: object


### Extract Date from datetime column

In [16]:
def convert_to_datetime(df, column_name):
    if column_name in df.columns:
        try:
            # df[column_name] = pd.to_datetime(df[column_name], format='%m/%d/%y %I:%M:%S %p')
            df['Date'] = df[column_name].dt.date
            return df
        except ValueError:
            print(f"Invalid datetime format in column '{column_name}':")
            print(df[column_name])
    else:
        print(f"Column '{column_name}' not found in the DataFrame.")
    return df

final_sheet = convert_to_datetime(final_sheet, 'datetime')
final_sheet['Date']

AttributeError: 'NoneType' object has no attribute 'columns'

### Graveyard

In [ ]:
def convert_to_datetime(df, column_name):
    if column_name in df.columns:
        if column_name == 'datetime':
            try:
                df['Date'] = df[column_name].dt.date
                # df[column_name] = pd.to_datetime(df[column_name], format='%m/%d/%y %I:%M:%S %p')
                return df
            except ValueError:
                print(f"Invalid datetime format in column {column_name}:")
                print(df[column_name])
        else:
            df[column_name] = pd.to_datetime(df[column_name], format='%m/%d/%y %I:%M:%S %p')
            return df
    else:
        print(f"Column '{column_name} not found in Dataframe.")
        return None
    
final_sheet = convert_to_datetime(final_sheet, 'datetime')

# print(final_sheet)